In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# DX vs Finclips

## Blank Correction & Lipid Nomalization

In [ ]:
tagToId = pd.read_excel('muslce,fin clips dioxcin,furans.xlsx')
finclip = pd.read_excel('muslce,fin clips dioxcin,furans.xlsx', sheet_name= 'Fin clips')
rl_nd = finclip.loc[finclip.iloc[:, -6] == "ND", finclip.columns[-4]].dropna()

finclip.loc[rl_nd.index, finclip.columns[-5]] = rl_nd.to_numpy('float32')/np.sqrt(2)

c = finclip.columns
c = c[~c.str.contains('Unnamed')]
finclip = finclip.iloc[5:, [0]].join(finclip.iloc[5:, 2::3])
finclip.columns = c.astype('str')
id_to_name_dict = dict(zip(tagToId['Tag name'], tagToId['Fish ID']))
id_to_name_dict.update({"13-Trembleur":"13"})
finclip.rename(columns=id_to_name_dict, inplace=True)
finclip.insert(1, 'tag', 'finclip')
finclip.columns = finclip.columns.map(lambda x: str(x))
lipid_id = finclip.iloc[:, 0].str.contains('lipid', case=False)
id = finclip.loc[~lipid_id, finclip.columns[-2]].dropna().index
finclip.loc[id, finclip.columns[2:-2]] -= finclip.loc[id, finclip.columns[-2]].to_numpy('float')[:, np.newaxis]
finclip = finclip.iloc[:, :-2]
print(f'count of samples corrected to negative: {np.sum((~(finclip.iloc[:, 2:].isna() | (finclip.iloc[:, 2:] > 0)).to_numpy()))}')
print(f'position of corrected samples: {np.where((~(finclip.iloc[:, 2:].isna() | (finclip.iloc[:, 2:] > 0))).to_numpy())}')
finclip.where((finclip.iloc[:, 2:]>0) | (finclip.iloc[:, 2:].isna()), 0, inplace = True)
finclip

finclip_l = finclip.copy()

lipid_id = np.arange(len(finclip_l.CLIENT_ID))[finclip_l.CLIENT_ID.str.contains('Lipid', case = False)][0]
finclip_l.iloc[lipid_id, 2:] /= 100
id = list(range(finclip_l.shape[0]))
id.pop(lipid_id)
finclip_l.iloc[id, 2:] /= finclip_l.iloc[lipid_id, 2:]
finclip_l

In [ ]:
muscle = pd.read_excel('muslce,fin clips dioxcin,furans.xlsx', sheet_name= 'Muslce')
rl_nd = muscle.loc[muscle.iloc[:, -6] == "ND", muscle.columns[-4]].dropna()
muscle.loc[rl_nd.index, muscle.columns[-5]] = rl_nd.to_numpy('float32')/np.sqrt(2)

c = muscle.columns
c = c[~c.str.contains('Unnamed')]
muscle = muscle.iloc[5:, [0]].join(muscle.iloc[5:, 2::3])
c = list(c.str.removeprefix("Nechako mort #"))
muscle.columns = c
muscle.insert(1, 'tag', 'muscle')

lipid_id = muscle.iloc[:, 0].str.contains('lipid', case=False)
id = muscle.loc[~lipid_id, muscle.columns[-2]].dropna().index
muscle.loc[id, muscle.columns[2:-2]] -= muscle.loc[id, muscle.columns[-2]].to_numpy('float')[:, np.newaxis]
muscle = muscle.iloc[:, :-2]
print(f'count of samples corrected to negative: {np.sum((~(muscle.iloc[:, 2:].isna() | (muscle.iloc[:, 2:] > 0)).to_numpy()))}')
print(f'position of corrected samples: {np.where((~(muscle.iloc[:, 2:].isna() | (muscle.iloc[:, 2:] > 0))).to_numpy())}')
muscle.where((muscle.iloc[:, 2:]>0) | (muscle.iloc[:, 2:].isna()), 0, inplace = True)


muscle_l = muscle.copy()

lipid_id = np.arange(len(muscle_l.CLIENT_ID))[muscle_l.CLIENT_ID.str.contains('Lipid', case = False)][0]
muscle_l.iloc[lipid_id, 2:] /= 100
id = list(range(muscle_l.shape[0]))
id.pop(lipid_id)
muscle_l.iloc[id, 2:] /= muscle_l.iloc[lipid_id, 2:]
muscle_l

blankcorrect_table = pd.concat([finclip, muscle], axis = 0)

blankcorrect_table['CLIENT_ID'] = blankcorrect_table["CLIENT_ID"].astype("category")
blankcorrect_table['CLIENT_ID'] = blankcorrect_table['CLIENT_ID'].cat.set_categories(finclip['CLIENT_ID'])
blankcorrect_table.sort_values(['CLIENT_ID'], inplace = True)
blankcorrect_table.reset_index(drop=True, inplace=True)
blankcorrect_table

### lipid nomalized

In [ ]:
lipid_table = pd.concat([finclip_l, muscle_l], axis = 0)
lipid_table.sort_values(['CLIENT_ID'], inplace = True)
lipid_table.reset_index(drop=True, inplace=True)
lipid_table.drop(index=[0, 1], inplace=True)
lipid_table.drop(index = lipid_table.index[lipid_table.CLIENT_ID.str.contains('WHO', case = False)], inplace=True)
lipid_table.drop(index = lipid_table.index[lipid_table.CLIENT_ID.str.contains('Total', case = False)], inplace=True)
lipid_table.drop(index = lipid_table.index[lipid_table.CLIENT_ID.str.contains('recovery', case = False)], inplace=True)
lipid_table.head()

In [ ]:
t= lipid_table
width = 0.2
for r in t["CLIENT_ID"].unique():
    plt.figure(facecolor='white')
    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "muscle"), t.columns[2:]].dropna(axis=1)
    x = np.arange(len(temp.columns)) - (width/2)
    plt.xticks(np.arange(len(temp.columns)), temp.columns)
    y = temp.to_numpy("float")[0]

    plt.bar(x,y, width=width, label = "muscle")


    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "finclip"), temp.columns]
    
    y = temp.to_numpy("float")[0]
    
    plt.bar(x + width,y, width=width, label = "finclip")
    plt.title(f'{r}')
    plt.legend()
    


In [ ]:
t= lipid_table

for r in t["CLIENT_ID"].unique():
    plt.figure(facecolor='white')
    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "muscle"), t.columns[2:]].dropna(axis=1)
    x = np.arange(len(temp.columns)) 
    plt.xticks(np.arange(len(temp.columns)), temp.columns)
    y = temp.to_numpy("float")[0]
    plt.xlabel("fish ID")
    plt.ylabel("pg/g lw")
    plt.scatter(x,y, label = "muscle")
    

    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "finclip"), temp.columns]
    
    y = temp.to_numpy("float")[0]
    
    plt.scatter(x,y, label = "finclip")
    plt.title(f'{r}')
    plt.legend()

In [ ]:
t= lipid_table
x = np.arange(len(t.columns[2:]))*2
width = 0.8
plt.figure(facecolor='white', figsize=(10, 10))
plt.xticks(np.r_[x+(width/2), x-(width/2)], np.r_[t.columns[2:]+"-finclip", t.columns[2:]+"-muscle"], rotation = 60, ha = 'right')
cmap = plt.get_cmap('Spectral')
z_c = len(t['CLIENT_ID'].unique())
for i, r in enumerate(t["CLIENT_ID"].unique()):
    color = cmap(i/z_c)
    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "muscle"), t.columns[2:]]
    y = temp.to_numpy("float")[0]
    plt.bar(x - (width/2),y, width=width, color = color)

    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "finclip"), temp.columns]
    y = temp.to_numpy("float")[0]
    plt.bar(x + width/2, y, width=width, label = f"{r}", color = color)
    
    plt.title('overall')
    plt.legend(bbox_to_anchor = (1, 1))
    


### each individual fish

In [ ]:
t = lipid_table
width = 0.3
for c in t.columns[2:]:
    plt.figure(facecolor='white', figsize = (8, 6))
    temp = t[c].dropna()
    # temp = temp.loc[temp != 0]
    temp = t.loc[temp.index, :]
    x = t.loc[temp.index, 'CLIENT_ID'].unique()
    xx = np.arange(x.shape[0])
    plt.xticks(xx, labels = x, rotation = 70, ha = 'right')
    muscle_id = [i in temp.loc[temp['tag'] == 'muscle', "CLIENT_ID"].to_list() for i in x]
    finclip_id = [i in temp.loc[temp['tag'] == 'finclip', "CLIENT_ID"].to_list() for i in x]
    plt.bar(xx[muscle_id] + width/2, temp.loc[temp.tag == 'muscle', c].to_numpy('float'), width = width, color = 'blue', label = 'muscle')
    plt.bar(xx[finclip_id] - width/2, temp.loc[temp.tag == 'finclip', c].to_numpy('float'), width = width, color = 'orange', label = 'finclip')
    plt.title(f'fish #{c}')
    plt.legend();

# PCBs vs Finclips

In [ ]:
muscle = pd.read_excel('MusclePCB.xlsx', sheet_name= 'DATA')

rlnd = muscle.loc[muscle.iloc[:,-6] == "ND", muscle.columns[-4]].dropna() #reporting limits who are NDs
muscle.loc[rlnd.index, muscle.columns[-5]] = rlnd.to_numpy('float32')/np.sqrt(2) #reporting limts /sqrt(2) instead of ND in Blank column

c = muscle.columns[~muscle.columns.str.contains("unnamed", case = False)]

muscle = muscle.iloc[:, [0] + list(range(2,len(muscle.columns), 3))] #all needed columns, removing extras
muscle.columns = c.str.removeprefix("Nechako mort #")
muscle.drop(index = [0,1,2,3,4], inplace = True) #deleting first 4 rows
muscle.drop(columns = 'Spiked Matrix (102)', inplace = True) #deleting Spiked Matrix (102)

muscle.set_index('CLIENT_ID', inplace = True) #change client_id column to index, it is not column anymore, cuase we want to change it to float
muscle = muscle.astype("float") #all table is now float
muscle.reset_index(inplace = True)
muscle.insert(1,"tag", "muscle") 


#blank correction: 

muscle.loc[~muscle["Lab Blank (101)"].isna(), muscle.columns[2:-1]] -= muscle.loc[~muscle["Lab Blank (101)"].isna(), muscle.columns[-1]].to_numpy('float')[:,np.newaxis] #columnwise

#lipid normalization:

lipid= muscle.loc[muscle.CLIENT_ID.str.contains("lipid", case = False), :]
id = muscle.index.to_numpy() != lipid.index.to_numpy() 
muscle.loc[id,muscle.columns[2::]]/= muscle.loc[~id, muscle.columns[2::]].to_numpy('float')

#muscle.where((muscle.iloc[:, 2:]>0) | (muscle.iloc[:, 2:].isna()), 0, inplace = True) #change - to 0
muscle.iloc[:, 2::] = muscle.iloc[:,2::].where((muscle.iloc[:,2::]>0) | (muscle.iloc[:, 2::].isna()), 0) #change - to 0
muscle.drop(index = muscle.index[muscle.CLIENT_ID.str.contains('recovery', case = False)], inplace=True)
muscle.drop(index = muscle.index[muscle.CLIENT_ID.str.contains('WHO')], inplace=True)
muscle.drop(columns='Lab Blank (101)', inplace = True)
muscle


In [ ]:
finclip = pd.read_excel("./generated excels/pcb.xlsx", sheet_name="lipid_normalized")
finclip.rename(columns={finclip.columns[0]: 'CLIENT_ID'}, inplace=True)
finclip.columns = finclip.columns.str.removesuffix("-Trembleur")
finclip.insert(1, 'tag', 'finclip')
finclip.head()

In [ ]:
pcb_table = pd.concat([finclip, muscle], axis = 0)
pcb_table.reset_index(drop=True, inplace=True)
pcb_table.head()

In [ ]:
pcb_table.drop(index = pcb_table.index[pcb_table.CLIENT_ID.str.contains('Total', case = False)], inplace=True)
pcb_table.drop(index = pcb_table.index[pcb_table.CLIENT_ID.str.contains('Lipid', case = False)], inplace = True)
pcb_table.head()

In [ ]:
t = pcb_table
x = np.arange(len(t.columns[2:]))*2
width = 0.8
plt.figure(facecolor='white', figsize=(10, 10))
plt.xticks(np.r_[x+(width/2), x-(width/2)], np.r_[t.columns[2:]+"-finclip", t.columns[2:]+"-muscle"], rotation = 60, ha = 'right')
cmap = plt.get_cmap('Spectral')
z_c = len(t['CLIENT_ID'].unique())
for i, r in enumerate(t["CLIENT_ID"].unique()):
    color = cmap(i/z_c)
    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "muscle"), t.columns[2:]]
    y = temp.to_numpy("float")[0]
    plt.bar(x - (width/2),y, width=width, color = color)

    temp = t.loc[(t.CLIENT_ID == r) & (t.tag == "finclip"), temp.columns]
    y = temp.to_numpy("float")[0]
    plt.bar(x + width/2, y, width=width, label = f"{r}", color = color)
    plt.title('overall')
    plt.legend(bbox_to_anchor = (1, 1), ncol = 4)